# import

In [55]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
assert gensim.models.word2vec.FAST_VERSION > -1
from gensim.models import Word2Vec
import datetime
import requests
import re
from bs4 import BeautifulSoup

# Data 준비하기

In [56]:
df_contest = pd.read_csv('contest_process.csv') # 대외활동 데이터
id_to_name = pd.Series(df_contest.contest_name.values, index = df_contest.id.values)
name_to_id = pd.Series(df_contest.id.values, index = df_contest.contest_name).to_dict()


df_contest.head(5)

,Unnamed: 0,id,contest_name,company_type,institution,target,rewards,app_period,contest_benefit,fields,...,대학생,직장인/일반인,청소년,benefit_기타,상장 수여,실제상용화,인턴/정규직채용,입사시 가산점,전시기회,해외연수
0,0,100024,제26회 서울환경작품공모전,비영리단체/협회/재단,(사)한국로하스협회,대상 제한 없음,1940만 원,22.8.16 ~ 22.9.15,"benefit_기타, 상장 수여","사진/영상/UCC, 기획/아이디어",...,0,0,0,1,1,0,0,0,0,0
1,1,100035,"2022년 제10회 아트챌린저 공모, 세계미술작가교류협회 선정 작가",비영리단체/협회/재단,세계미술작가교류협회 세미갤러리,대상 제한 없음,-,22.9.23 ~ 상시모집,"benefit_기타, 상장 수여","디자인/순수미술/공예, 사진/영상/UCC",...,0,0,0,1,1,0,0,0,0,0
2,2,100050,2022 문화도시 조성사업 로컬콘텐츠 지원사업 참여자 모집 공고,비영리단체/협회/재단,수원 문화재단,대상 제한 없음,300만 원,22.8.8 ~ 22.8.24,benefit_기타,"기획/아이디어, 디자인/순수미술/공예",...,0,0,0,1,0,0,0,0,0,0
3,3,100056,2022년 생명사랑 로고송 댄스 챌린지 경연대회,공공기관/공기업,전라남도광역정신건강복지센터,대상 제한 없음,950만 원,22.8.16 ~ 22.9.8,"benefit_기타, 상장 수여","사진/영상/UCC, 예체능/패션",...,0,0,0,1,1,0,0,0,0,0
4,4,100059,천하제일 미에로화이바 그림대회,중견기업,현대약품,대상 제한 없음,650만 원,22.8.16 ~ 22.9.11,benefit_기타,디자인/순수미술/공예,...,0,0,0,1,0,0,0,0,0,0


# 필요없는 열제거

In [57]:
df_contest=df_contest.drop(["company_type"],axis='columns')
df_contest=df_contest.drop(["Unnamed: 0"],axis='columns')
df_contest=df_contest.drop(["institution"],axis='columns')
df_contest=df_contest.drop(["contest_benefit"],axis='columns')
df_contest=df_contest.drop(["fields"],axis='columns')
df_contest=df_contest.drop(["target"],axis='columns')
df_contest=df_contest.drop(["rewards"],axis='columns')
df_contest=df_contest.drop(["app_period"],axis='columns')
df_contest=df_contest.drop(["시상규모(만원)"],axis='columns')
df_contest.head(3)

,id,contest_name,시상 규모,건축/건설/인테리어,공모분야_기타,과학/공학,광고/마케팅,기타,기획/아이디어,네이밍/슬로건,...,대학생,직장인/일반인,청소년,benefit_기타,상장 수여,실제상용화,인턴/정규직채용,입사시 가산점,전시기회,해외연수
0,100024,제26회 서울환경작품공모전,대,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
1,100035,"2022년 제10회 아트챌린저 공모, 세계미술작가교류협회 선정 작가",시상규모미정,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
2,100050,2022 문화도시 조성사업 로컬콘텐츠 지원사업 참여자 모집 공고,중,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


# train set, test set 분리

In [58]:
#df_contest_train, df_contest_test = train_test_split(df_contest, random_state = 15688, test_size=0.30)

df_contest_train = df_contest




In [59]:
df_contest_train.head(3)

,id,contest_name,시상 규모,건축/건설/인테리어,공모분야_기타,과학/공학,광고/마케팅,기타,기획/아이디어,네이밍/슬로건,...,대학생,직장인/일반인,청소년,benefit_기타,상장 수여,실제상용화,인턴/정규직채용,입사시 가산점,전시기회,해외연수
0,100024,제26회 서울환경작품공모전,대,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
1,100035,"2022년 제10회 아트챌린저 공모, 세계미술작가교류협회 선정 작가",시상규모미정,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
2,100050,2022 문화도시 조성사업 로컬콘텐츠 지원사업 참여자 모집 공고,중,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


# 같은 대외활동끼리 묶기

In [60]:
# 비교할 때 제외할 열들의 이름 리스트
def data_splitter(df):
    exclude_cols = ['id','contest_name']

    grouped_cols = [col for col in df_contest.columns if col not in exclude_cols]  # 제외할 열을 제외한 열들의 리스트

    df_grouped = df_contest.groupby(grouped_cols).apply(lambda x: x.id.tolist())
    
    # 결과값을 str 타입으로 저장한 이중 배열
    array_of_groups = df_grouped.apply(lambda x: [str(val) for val in x]).tolist()
    return array_of_groups

In [61]:
pd.options.mode.chained_assignment = None
contest_ls = data_splitter(df_contest_train)

# 데이터 랜덤으로 섞고, 유효한 데이터 골라내기

In [62]:
for contest in contest_ls:
    random.shuffle(contest)

In [63]:
filtered_array = [group for group in contest_ls if len(group) > 1]

In [64]:
len(filtered_array)

553

In [65]:
print(filtered_array)

[['132434', '132436'], ['123201', '123157'], ['104022', '103604'], ['125380', '133722', '134571'], ['116535', '109800'], ['104474', '125006', '100655'], ['103063', '100166'], ['114216', '117214'], ['101197', '109461', '113108', '128249', '115821', '109291', '109281', '112222', '109091', '129724', '122967'], ['131557', '108476', '126320'], ['135139', '126931'], ['125165', '135364', '133720', '103638', '107598', '106227', '127326'], ['127321', '114561'], ['121780', '132802', '101739'], ['116456', '100479'], ['133034', '125331'], ['113105', '128524', '130224', '105086'], ['131898', '127344'], ['116534', '122961'], ['134184', '100130', '105088'], ['133036', '125883', '100672', '134907'], ['109084', '109277'], ['100719', '103803'], ['101757', '108481'], ['119589', '134922', '132222', '103387', '104506', '104671', '115959'], ['134915', '132862', '131182'], ['113234', '100481', '120210', '107582', '133245', '127106', '127316'], ['129987', '100685', '134182', '128692'], ['126729', '127105'], [

# Model 생성 및 학습

In [66]:
model = Word2Vec(
    sentences = filtered_array, # 전처리된 리스트를 파라미터로 사용
    epochs = 70, # epoch
    min_count = 1, # 
    vector_size = 100, # hidden layer 사이즈
    sg = 1, # skip-gram을 사용하면 1
    hs = 0, # negative sampling을 사용하면 0
    negative = 5, # 0 이상일 경우 negative sampling을 사용함.
    window=5,  # 주변 단어 창 크기
    workers=4  # 사용할 CPU 스레드 수
)

model.save('item2vec_contest')

In [67]:
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
model = Word2Vec.load('item2vec_contest')
word_vectors = model.wv

# 추천

In [68]:
def recommender(positive_list=None, negative_list=None, topn=20):
    recommend_contest_ls = []
    
    for contestId, prob in model.wv.most_similar_cosmul(positive=positive_list, negative=negative_list, topn=topn):
        recommend_contest_ls.append(contestId)
    return recommend_contest_ls

In [69]:
reco = [133040]
reco_str = [str(num) for num in reco]
recommandations = recommender(positive_list=reco_str, topn=5)
reco_int = [int(value) for value in recommandations] # str -> int
print(reco_int)

[127329, 131713, 106767, 106408, 119593]


In [70]:
# 입력 데이터
filtered_contest_names = df_contest.loc[df_contest['id'].isin(reco), 'contest_name'].tolist()
print(filtered_contest_names)

['경북 홍보 유튜브 쇼츠 영상 공모전 (경북 숏 타임)']


In [71]:
print(model.wv.most_similar(["133040"]))

[('127329', 0.9868972301483154), ('131713', 0.9852855801582336), ('106767', 0.9850942492485046), ('106408', 0.9835209846496582), ('119593', 0.9797326326370239), ('125756', 0.9796921014785767), ('131565', 0.9778892397880554), ('104518', 0.9763807654380798), ('107798', 0.9761024713516235), ('108310', 0.9757652878761292)]
